# Time Series Model

In [40]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import holidays
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import plotly.graph_objects as go

Monterrey = pd.read_csv("Files/Datos_2022_2023_CENTRO_limpios.csv", parse_dates=[0], index_col = "date")
daily_Monterrey = Monterrey.resample('D').mean()

## Functions

In [41]:
import sys
sys.path.append("Functions")

import Feature_Engineering
import Model

### Feature Engineering

In [42]:
from Feature_Engineering import add_holiday_feature
from Feature_Engineering import add_weekend_feature
from Feature_Engineering import add_cyclic_feature

### Model

In [43]:
from Model import scale_dataframe
from Model import split_data

## Data lecture and feature engineering

In [44]:
daily_Monterrey, festivos_df = add_holiday_feature(daily_Monterrey, 2022, 2023, country='Mexico')
daily_Monterrey = add_weekend_feature(daily_Monterrey)
daily_Monterrey = add_cyclic_feature(daily_Monterrey, "2022-01-03", 37, column_name="is_37_day_cycle")

# CO

## Scaling

In [45]:
X_scaled, y_scaled, scaler_X, scaler_y = scale_dataframe(daily_Monterrey, ["PRS", "RAINF", "RH", "SO2", "SR", "TOUT", "WSR", "WDR"], "CO", feature_range=(0, 1), return_scalers=True)

## Data training, Validation and testing sets

In [46]:
# Número de días para testeo y validación
n_test = 31
n_val = 31

X_train, y_train, X_val, y_val, X_test, y_test = split_data(X_scaled, y_scaled, n_test, n_val)

## Modelo

In [48]:
# Reformatear los datos para LSTM (las redes LSTM esperan datos en 3D)
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))  # (muestras, pasos de tiempo, características)
X_val_lstm = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Crear y compilar el modelo LSTM
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dense(1)
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Entrenar el modelo
history = model.fit(X_train_lstm, y_train, epochs=20, batch_size=16, validation_data=(X_val_lstm, y_val))

# Evaluar el modelo en el conjunto de test
loss = model.evaluate(X_test_lstm, y_test)
print(f"Pérdida final en el conjunto de test: {loss}")

# Visualización de la evolución de la pérdida durante el entrenamiento
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.title('Evolución de la pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida (MSE)')
plt.legend()
plt.show()

AttributeError: 'DataFrame' object has no attribute 'reshape'

## Predicciones

In [ ]:
predictions = model.predict(X_test_lstm)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step


Visualización

In [ ]:
# Desescalar las predicciones y los valores reales
predictions_original = scaler_y.inverse_transform(predictions)  # Desescalar las predicciones
y_test_original = scaler_y.inverse_transform(y_test)  # Desescalar los valores reales

# Crear un DataFrame con las fechas, los valores reales y las predicciones desescaladas
fechas_test = daily_Monterrey.index[-n_test:]  # Ajusta esto a tu conjunto de fechas
df_predictions = pd.DataFrame({
    'Fecha': fechas_test,
    'Valor Real': y_test_original.flatten(),  # Desescalar y convertir a 1D
    'Predicciones': predictions_original.flatten()  # Desescalar y convertir a 1D
})
df_predictions.set_index('Fecha', inplace=True)

# Graficar los resultados usando Plotly
fig = go.Figure()

# Graficar los valores reales
fig.add_trace(go.Scatter(
    x=df_predictions.index,
    y=df_predictions['Valor Real'],
    name='Valor Real',
    mode='lines',
    line=dict(color='blue')
))

# Graficar las predicciones
fig.add_trace(go.Scatter(
    x=df_predictions.index,
    y=df_predictions['Predicciones'],
    name='Predicciones',
    mode='lines',
    line=dict(color='red', dash='dash')
))

# Configuración de la gráfica
fig.update_layout(
    title='Comparación entre Predicciones y Valores Reales (Escala Original)',
    xaxis_title='Fecha',
    yaxis_title='Valor CO',
    template='plotly_dark',
    legend=dict(x=0.8, y=0.9)
)

# Mostrar la gráfica
fig.show()